# CPU or GPU 

In [ ]:
import tensorflow as tf
# # Configuración de TensorFlow para usar la GPU
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#     except RuntimeError as e:
#         print(e)

# Configuración de TensorFlow para usar la CPU
tf.config.set_visible_devices([], 'GPU')

# LIBRARIES 

In [ ]:
# Library EEG
import mne

# Library System 
import os
import numpy as np
from tqdm import tqdm
import time

# Library Statistics
import scipy.stats
import random
from scipy.stats import entropy
from scipy.fft import fft
from scipy.signal import welch

# Library GAN
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization, Reshape, Conv2DTranspose, Conv2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Library Graph
import matplotlib.pyplot as plt
import seaborn as sns

# No Warning
import warnings
warnings.filterwarnings("ignore")
mne.set_log_level("ERROR")

In [ ]:
os.getcwd()

# STATISTICS CALCULATES

In [ ]:
# Tiempo de ejecución del Código
start_time = time.time()

emotion='POSITIVE' # 'NEGATIVE' 'NEUTRAL'
emotion_folder =f'C://Users//macka//TFM_WD//ORI//SEED_GER//01-EEG-raw//SEED_GER//{emotion}'
file_list = [file for file in os.listdir(emotion_folder) if file.endswith('.fif')]

# Statistics List
means = []
covariances = []
entropies = []
kurtoses = []
skewnesses = []
standard_deviations = []

for file in tqdm(file_list, desc="Processing files", leave=False):
    file_path = os.path.join(emotion_folder, file)
    eeg_data = mne.io.read_raw_fif(file_path, preload=False)
    
    data = eeg_data.get_data()
    means.append(np.mean(data, axis=1))
    covariances.append(np.cov(data))
    
    def spectral_entropy(data, sfreq):
        # Calcular la densidad espectral de potencia usando la función de Welch
        freqs, psd = welch(data, sfreq, nperseg=sfreq)
        # Normalizar la densidad espectral de potencia
        psd_norm = np.divide(psd, np.sum(psd))
        # Calcular la entropía espectral (entropía de Shannon)
        se = -np.sum(psd_norm * np.log2(psd_norm))
        return se

    # Calcular la entropía espectral para cada canal
    entropies = []
    for channel_data in data:
        se = spectral_entropy(channel_data, eeg_data.info['sfreq'])
        entropies.append(se)

    kurtoses.append(scipy.stats.kurtosis(data, axis=1))
    skewnesses.append(scipy.stats.skew(data, axis=1))
    standard_deviations.append(np.std(data, axis=1))
    
# Media de las estadisticas de todos los archivos
mean_mean = np.mean(means, axis=0)
mean_covariance = np.mean(covariances, axis=0)
mean_entropy = np.mean(entropies, axis=0)
mean_kurtosis = np.mean(kurtoses, axis=0)
mean_skewness = np.mean(skewnesses, axis=0)
mean_standard_deviation = np.mean(standard_deviations, axis=0)

elapsed_time = time.time() - start_time
print(f"{elapsed_time:.2f} segundos")

## PLOT STATISTICS RESULTS

### AUX FUNCTION.

In [ ]:
def plot_bargrid(stat_list, stat_name, channel_names):
    n_files = len(stat_list)
    n_channels = len(channel_names)

    fig, axs = plt.subplots(n_files, 1, figsize=(15, n_files * 5), sharex=False)
    fig.suptitle(stat_name)

    for i, (file_stat, ax) in tqdm(enumerate(zip(stat_list, axs)), desc="Generating graphics..", leave=False):
        sns.barplot(x=channel_names, y=file_stat, ax=ax)
        ax.set_ylabel(file_list[i])
        ax.set_xticklabels(channel_names, rotation=90, fontsize=8)
        ax.grid(True)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

def plot_covariances(cov_list, channel_names):
    n_files = len(cov_list)

    fig, axs = plt.subplots(n_files, 1, figsize=(15, n_files * 5), sharex=False)
    fig.suptitle("Covariances")

    for i, (file_cov, ax) in tqdm(enumerate(zip(cov_list, axs)), desc="Generating graphics..", leave=False):
        sns.heatmap(file_cov, ax=ax, cmap='coolwarm', xticklabels=channel_names, yticklabels=channel_names)
        ax.set_ylabel(file_list[i])

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

###  GRAPHICS CHOOSE: MEAN, CONVARIANCE, SPECTRAL-ENTROPY, SKEWNESSES, STD

In [ ]:
start_time = time.time()
channel_names = eeg_data.ch_names

# Distribución de las listas de estadisticas.

# plot_bargrid(means, "Means", channel_names) # - MEDIA
plot_covariances(covariances, channel_names) # -  COVARIANZA
# plot_bargrid(entropies, "Entropies", channel_names) # - ENTROPÍA ESPECTRAL
# plot_bargrid(kurtoses, "Kurtoses", channel_names) # - KURTOSES
# plot_bargrid(skewnesses, "Skewnesses", channel_names) # - SIMETRÍA
# plot_bargrid(standard_deviations, "Standard Deviations", channel_names) # - DESV. STANDARD

# print_statistics(means, "Means")
# print_statistics(covariances, "Covariances")
# print_statistics(entropies, "Entropies")
# print_statistics(kurtoses, "Kurtoses")
# print_statistics(skewnesses, "Skewnesses")
# print_statistics(standard_deviations, "Standard Deviations")

# def print_statistics(stat_list, stat_name):
#     print(f"{stat_name}:\n")
#     for i, file_stat in enumerate(stat_list):
#         print(f"Archivo {i + 1}:")
#         for channel_name, channel_stat in zip(channel_names, file_stat):
#             print(f"{channel_name}: {channel_stat}")
#         print()



elapsed_time = time.time() - start_time
print(f"{elapsed_time:.2f} segundos")

**Media:** La media de los valores de cada canal puede indicar si hay un sesgo en los datos. Si las medias son similares en todos los canales, esto podría sugerir que los datos son consistentes entre los canales. Si un canal tiene una media significativamente diferente, podría ser un indicador de problemas en ese canal, como ruido o artefactos.

**Covarianza:** La covarianza mide la relación entre los canales de EEG. Valores altos de covarianza indican que los canales están relacionados, lo que podría sugerir que comparten información o ruido común. Por otro lado, valores bajos indican que los canales son independientes entre sí. La matriz de covarianza también puede ser útil para la reducción de dimensiones y el análisis de componentes principales.

**La entropía espectral** es una medida que cuantifica la incertidumbre o complejidad en la distribución de energía de una señal en el dominio de la frecuencia. En el caso de las señales de EEG, la entropía espectral se utiliza para estimar la distribución del espectro de potencia de las señales a lo largo de diferentes bandas de frecuencia.

La entropía espectral se calcula utilizando la **entropía de Shannon**, que se basa en la teoría de la información. La entropía de Shannon mide la cantidad de información o incertidumbre en una distribución de probabilidad. En el caso de la entropía espectral, primero se calcula la densidad espectral de potencia (PSD) de la señal de EEG utilizando técnicas como el método de Welch. Luego, se normaliza la PSD para obtener una distribución de probabilidad y, finalmente, se aplica la entropía de Shannon a esta distribución normalizada.

Calcular la entropía espectral en el análisis de EEG es importante por varias razones:

- **Caracterización de la señal:** La entropía espectral permite describir la complejidad de la señal de EEG en términos de su distribución de energía a lo largo de diferentes frecuencias. Esto puede ser útil para identificar patrones en la actividad cerebral y para comparar diferentes estados cerebrales.

- **Detección de cambios en el estado cerebral:** Cambios en la entropía espectral pueden reflejar cambios en la dinámica cerebral, como transiciones entre diferentes estados de conciencia o cambios en la actividad cerebral debido a estímulos externos.

- **Extracción de características para el aprendizaje automático:** La entropía espectral puede ser utilizada como una característica en algoritmos de aprendizaje automático para clasificar señales de EEG según diferentes estados o condiciones, como emociones o tareas cognitivas.

- **Análisis de la conectividad funcional:** La entropía espectral puede ser utilizada para estudiar la conectividad funcional entre diferentes regiones cerebrales, al comparar la complejidad espectral en diferentes canales de EEG.

**Curtosis:** La curtosis mide la "cola pesada" o la concentración de valores extremos en una distribución. Valores altos de curtosis indican que hay una mayor concentración de valores extremos, mientras que valores bajos indican una menor concentración. Esto puede ser útil para identificar canales con actividad inusual o ruido impulsivo.

**Asimetría (Skewness):** La asimetría mide la falta de simetría en la distribución de los datos. Valores positivos de asimetría indican que hay una cola más larga en el lado derecho de la distribución, mientras que valores negativos indican que hay una cola más larga en el lado izquierdo. La asimetría puede ser útil para identificar distribuciones no simétricas de actividad cerebral que podrían estar relacionadas con procesos cognitivos o emocionales específicos.

**Desviación estándar:** La desviación estándar mide la dispersión o variabilidad de los datos. Valores altos de desviación estándar indican que los datos son más dispersos, mientras que valores bajos indican que los datos son más uniformes. La desviación estándar puede ser útil para identificar la consistencia en los datos de EEG entre ensayos y sujetos.

## ¿Que hacer con los canales o sujetos que se salen de la media? 

## GANs

In [ ]:
start_time = time.time()
# 1. Cargar y preparar los datos
emotion = 'POSITIVE'  # 'NEGATIVE' 'NEUTRAL'
emotion_folder = f'C://Users//macka//TFM_WD//ORI//SEED_GER//01-EEG-raw//SEED_GER//{emotion}'
file_list = [file for file in os.listdir(emotion_folder) if file.endswith('.fif')]

eeg_data_list = []
for file in file_list:
    file_path = os.path.join(emotion_folder, file)
    eeg_data = mne.io.read_raw_fif(file_path, preload=False)
    eeg_data_list.append(eeg_data.get_data())

# Determine the common length for interpolation
common_length = 10000  # You can adjust this value depending on your needs

interpolated_eeg_data_list = []

for eeg_data in eeg_data_list:
    num_channels = eeg_data.shape[0]
    original_length = eeg_data.shape[1]
    new_eeg_data = np.zeros((num_channels, common_length))

    for ch_idx in range(num_channels):
        x_original = np.linspace(0, 1, original_length)
        x_new = np.linspace(0, 1, common_length)
        new_eeg_data[ch_idx] = np.interp(x_new, x_original, eeg_data[ch_idx])

    interpolated_eeg_data_list.append(new_eeg_data)

eeg_data_tensor = np.stack(interpolated_eeg_data_list, axis=0)
eeg_data_tensor = (eeg_data_tensor - np.min(eeg_data_tensor)) / (np.max(eeg_data_tensor) - np.min(eeg_data_tensor)) * 2 - 1

# 2. Crear el modelo GAN
# Generador
# def build_generator(latent_dim):
#     input_layer = Input(shape=(latent_dim,))
#     x = Dense(128)(input_layer)
#     x = LeakyReLU(alpha=0.2)(x)
#     x = BatchNormalization()(x)
#     x = Dense(256)(x)
#     x = LeakyReLU(alpha=0.2)(x)
#     x = BatchNormalization()(x)
#     x = Dense(512)(x)
#     x = LeakyReLU(alpha=0.2)(x)
#     x = BatchNormalization()(x)
#     x = Dense(np.prod(eeg_data_tensor.shape[1:]), activation='tanh')(x)
#     output_layer = Reshape(eeg_data_tensor.shape[1:])(x)

#     return Model(input_layer, output_layer)
def build_generator(latent_dim):
    input_layer = Input(shape=(latent_dim,))
    
    x = Dense(256)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(1024)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(2048)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(np.prod(eeg_data_tensor.shape[1:]), activation='tanh')(x)
    output_layer = Reshape(eeg_data_tensor.shape[1:])(x)

    return Model(input_layer, output_layer)

# Discriminador
def build_discriminator():
    input_layer = Input(shape=eeg_data_tensor.shape[1:])
    x = Flatten()(input_layer)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.5)(x)
    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.5)(x)
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.2)(x)
    output_layer = Dense(1, activation='sigmoid')(x)

    return Model(input_layer, output_layer)

# GAN
def build_gan(generator, discriminator):
    z = Input(shape=(latent_dim,))
    generated_eeg = generator(z)
    validity = discriminator(generated_eeg)

    return Model(z, validity)

latent_dim = 100
generator = build_generator(latent_dim)
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
discriminator.trainable = False

gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# 3. Entrenar el GAN
# Aquí puedes agregar el código para entrenar el GAN utilizando tus datos de EEG preprocesados (eeg_data_tensor).
# Consulta este tutorial para entender cómo entrenar
# 3. Entrenar el GAN
epochs = 200
batch_size = 32
half_batch = int(batch_size / 2)

for epoch in range(epochs):
    # Seleccionar un batch aleatorio de EEG reales
    idx = np.random.randint(0, eeg_data_tensor.shape[0], half_batch)
    real_eegs = eeg_data_tensor[idx]

    # Generar un batch de EEG falsos
    noise = np.random.normal(0, 1, (half_batch, latent_dim))
    fake_eegs = generator.predict(noise)

    # Entrenar el discriminador
    real_loss = discriminator.train_on_batch(real_eegs, np.ones((half_batch, 1)))
    fake_loss = discriminator.train_on_batch(fake_eegs, np.zeros((half_batch, 1)))
    d_loss = 0.5 * np.add(real_loss, fake_loss)

    # Entrenar el generador
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

    # Imprimir el progreso
    print(f"Epoch {epoch}/{epochs} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}] [G loss: {g_loss}]")

elapsed_time = time.time() - start_time
print(f"El código tardó {elapsed_time:.2f} segundos en ejecutarse.")

**D_loss (Discriminator loss):** Es la pérdida del discriminador, que mide qué tan bien el discriminador puede distinguir entre datos reales y datos generados. Un valor bajo de D_loss indica que el discriminador tiene un buen rendimiento en la clasificación de los datos de entrada como reales o falsos. En el ejemplo proporcionado, D_loss es 0.0039080461574485525, lo que sugiere que el discriminador está haciendo un buen trabajo en este punto del entrenamiento.

**acc. (Discriminator accuracy):** Es la precisión del discriminador, que mide el porcentaje de datos de entrada que el discriminador clasifica correctamente como reales o falsos. En el ejemplo proporcionado, la precisión del discriminador es del 100%, lo que significa que el discriminador ha clasificado correctamente todos los datos de entrada en esta época.

**G_loss (Generator loss):** Es la pérdida del generador, que mide qué tan bien el generador puede engañar al discriminador generando datos falsos que se parezcan a los datos reales. Un valor alto de G_loss indica que el generador necesita mejorar su capacidad para generar datos que parezcan reales. En el ejemplo proporcionado, G_loss es 5.9058027267456055, lo que sugiere que el generador aún puede mejorar en la generación de datos que parezcan reales.

In [ ]:
def generate_artificial_eeg(generator, latent_dim):
    # Generar ruido aleatorio con la dimensión latente
    noise = np.random.normal(0, 1, (1, latent_dim))
    
    # Usar el generador para crear un EEG artificial a partir del ruido
    artificial_eeg = generator.predict(noise)
    
    return artificial_eeg

# Generar un EEG artificial
generated_eeg = generate_artificial_eeg(generator, latent_dim)

# # Opcional: Convertir el EEG artificial a su rango original de valores
# original_range_generated_eeg = (generated_eeg + 1) / 2 * (np.max(eeg_data_tensor) - np.min(eeg_data_tensor)) + np.min(eeg_data_tensor)


In [ ]:
import matplotlib.pyplot as plt

def plot_generated_eeg(generated_eeg, original_eeg_data, channel_names, sfreq=250):
    # Crear un objeto RawArray de MNE a partir del EEG generado
    info = mne.create_info(ch_names=channel_names, sfreq=sfreq, ch_types='eeg')
    raw_generated_eeg = mne.io.RawArray(generated_eeg, info)

    # Crear un objeto RawArray de MNE a partir del EEG original
    random_file = random.choice(file_list)
    random_file_path = os.path.join(emotion_folder, random_file)
    raw_original_eeg = mne.io.read_raw_fif(random_file_path, preload=True)
    eeg_data_array = raw_original_eeg.get_data()
    # Visualizar las señales de ambos EEG (original y generado)
    print("Original EEG:")
    eeg_data_array.plot(duration=12, scalings="auto", show=True)
    plt.show()

    print("Generated EEG:")
    raw_generated_eeg.plot(duration=12, scalings="auto", show=True)
    plt.show()

# Asumiendo que 'eeg_data_list' contiene los datos originales de EEG cargados anteriormente
# random_file = random.choice(file_list)
# random_file_path = os.path.join(emotion_folder, random_file)
# eeg_raw_sample = mne.io.read_raw_fif(random_file_path, preload=False)

# Visualizar la señal generada y una señal original de ejemplo
plot_generated_eeg(original_range_generated_eeg[0], eeg_data_list[0], channel_names)

In [ ]:
# 1. Cargar y preparar los datos
emotion = 'POSITIVE'  # 'NEGATIVE' 'NEUTRAL'
emotion_folder = f'C://Users//macka//TFM_WD//ORI//SEED_GER//01-EEG-raw//SEED_GER//{emotion}'
file_list = [file for file in os.listdir(emotion_folder) if file.endswith('.fif')]

eeg_data_list = []
for file in file_list:x
    file_path = os.path.join(emotion_folder, file)
    eeg_data = mne.io.read_raw_fif(file_path, preload=True)
    eeg_data_list.append(eeg_data.get_data())

# Determine the common length for interpolation
common_length = 3000  # You can adjust this value depending on your needs

interpolated_eeg_data_list = []

for eeg_data in eeg_data_list:
    num_channels = eeg_data.shape[0]
    original_length = eeg_data.shape[1]
    new_eeg_data = np.zeros((num_channels, common_length))

    for ch_idx in range(num_channels):
        x_original = np.linspace(0, 1, original_length)
        x_new = np.linspace(0, 1, common_length)
        new_eeg_data[ch_idx] = np.interp(x_new, x_original, eeg_data[ch_idx])

    interpolated_eeg_data_list.append(new_eeg_data)

eeg_data_tensor = np.stack(interpolated_eeg_data_list, axis=0)
eeg_data_tensor = (eeg_data_tensor - np.min(eeg_data_tensor)) / (np.max(eeg_data_tensor) - np.min(eeg_data_tensor)) * 2 - 1

# 2. Crear el modelo GAN
# Generador
# def build_generator(latent_dim):
#     input_layer = Input(shape=(latent_dim,))
#     x = Dense(128)(input_layer)
#     x = LeakyReLU(alpha=0.2)(x)
#     x = BatchNormalization()(x)
#     x = Dense(256)(x)
#     x = LeakyReLU(alpha=0.2)(x)
#     x = BatchNormalization()(x)
#     x = Dense(512)(x)
#     x = LeakyReLU(alpha=0.2)(x)
#     x = BatchNormalization()(x)
#     x = Dense(np.prod(eeg_data_tensor.shape[1:]), activation='tanh')(x)
#     output_layer = Reshape(eeg_data_tensor.shape[1:])(x)

#     return Model(input_layer, output_layer)
def build_generator(latent_dim):
    input_layer = Input(shape=(latent_dim,))
    
    x = Dense(256)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(1024)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(2048)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(np.prod(eeg_data_tensor.shape[1:]), activation='tanh')(x)
    output_layer = Reshape(eeg_data_tensor.shape[1:])(x)

    return Model(input_layer, output_layer)

# Discriminador
def build_discriminator():
    input_layer = Input(shape=eeg_data_tensor.shape[1:])
    x = Flatten()(input_layer)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.5)(x)
    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.5)(x)
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.2)(x)
    output_layer = Dense(1, activation='sigmoid')(x)

    return Model(input_layer, output_layer)

# GAN
def build_gan(generator, discriminator):
    z = Input(shape=(latent_dim,))
    generated_eeg = generator(z)
    validity = discriminator(generated_eeg)

    return Model(z, validity)

latent_dim = 100
generator = build_generator(latent_dim)
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
discriminator.trainable = False

gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# 3. Entrenar el GAN
# Aquí puedes agregar el código para entrenar el GAN utilizando tus datos de EEG preprocesados (eeg_data_tensor).
# Consulta este tutorial para entender cómo entrenar
# 3. Entrenar el GAN
epochs = 20000
batch_size = 32
half_batch = int(batch_size / 2)

for epoch in range(epochs):
    # Seleccionar un batch aleatorio de EEG reales
    idx = np.random.randint(0, eeg_data_tensor.shape[0], half_batch)
    real_eegs = eeg_data_tensor[idx]

    # Generar un batch de EEG falsos
    noise = np.random.normal(0, 1, (half_batch, latent_dim))
    fake_eegs = generator.predict(noise)

    # Entrenar el discriminador
    real_loss = discriminator.train_on_batch(real_eegs, np.ones((half_batch, 1)))
    fake_loss = discriminator.train_on_batch(fake_eegs, np.zeros((half_batch, 1)))
    d_loss = 0.5 * np.add(real_loss, fake_loss)

    # Entrenar el generador
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

    # Imprimir el progreso
    print(f"Epoch {epoch}/{epochs} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}] [G loss: {g_loss}]")
